# Import libraries

In [100]:
import sys
sys.path.append('modules')

# Import Custom Modules
import modules.source as src
import modules.target as tgt

### Setting local varibale

In [101]:
is_debugging = 1

wtb_1_any_ds_url   = "https://finance.yahoo.com/quote/"
wtb_2_any_ds_path  = "NVDA/history/?period1=1709544776&period2=1741080767&filter=history" 
wtb_3_any_ni_index = "0"

nm_target_schema = "tsl_psa_yahoo_exchange_rate"
nm_target_table  = "tsl_eur_x_usd"
 

## Extract web table from Yahoo Finance

In [102]:
# Import Custom Modules
from modules import session     as ss
from modules import secret      as sc 

# Import for web_table_anonymous_web
import pandas as pd
import time
from selenium.webdriver.common.by import By as by
from selenium                     import webdriver
from bs4                          import BeautifulSoup
from io                           import StringIO

In [103]:
# Initialize the WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Open the webpage
driver.get(wtb_1_any_ds_url + wtb_2_any_ds_path)

# Wait for the page to load (you might need to adjust the sleep time)
time.sleep(2)

try: # Find and click the "Accept Cookies" button (adjust the selector as needed)
    accept_button = driver.find_element(by.XPATH, '//button[text()="Alles accepteren"]')
    accept_button.click()

    # Wait for the page to load after accepting cookies
    time.sleep(2)

except Exception as e:
    # Code to handle any other exceptions
    print(f"An unexpected error occurred: {e}")        

# Get the page source after accepting cookies
page_source = driver.page_source

# Close the browser
driver.quit()

An unexpected error occurred: HTTPConnectionPool(host='localhost', port=59572): Read timed out. (read timeout=120)


In [104]:
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Create a StringIO object
table = StringIO()

# Find the table in the webpage (you might need to adjust the selector based on the webpage structure)
table.write(str(soup.find('table')))

# Read the table into a pandas DataFrame
pandas_df = pd.read_html(table.getvalue())[int(wtb_3_any_ni_index)]


C:\Users\mehme\AppData\Local\Temp\ipykernel_11128\2596355597.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pandas_df = pd.read_html(table.getvalue())[int(wtb_3_any_ni_index)]


In [113]:
# Initialize Spark session
spark = ss.getSparkSession("WebTableToSparkDataFrame")

In [114]:
# Convert the pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(pandas_df)


In [115]:
# Import Custom Modules
from modules import credentials as sa
from modules import session     as ss 

In [ ]:
 # Database credentials
ds_url  = f"jdbc:sqlserver://{sa.target_db['server']};"
ds_url += f"databaseName={sa.target_db['database']};"
ds_url += f"encrypt=false;trustServerCertificate=false"


In [117]:
# Truncate Target Table
spark = ss.getSparkSession("TruncateTableFunction")

In [131]:
nm_spark_table = f"{nm_target_schema}.{nm_target_table}"
print(nm_spark_table)

tsl_psa_yahoo_exchange_rate.tsl_eur_x_usd


In [ ]:
#As it was mentioned in other posts, there are few approaches to that:

#Use JDBC directly to execute your code. In PySpark, you will need to go through JVM gateway with something like this (from this answer):
tx_truncate_table = f"TRUNCATE TABLE {nm_target_schema}.{nm_target_table}"
driver_manager = spark._sc._gateway.jvm.java.sql.DriverManager
connection = driver_manager.getConnection(ds_url, sa.target_db['username'], sa.target_db['password'])
connection.prepareCall(tx_truncate_table).execute()
connection.close()


In [ ]:
tx_delete_query = f"DELETE FROM {nm_spark_table}"
print(tx_delete_query)
print(ds_url)

spark.read.format("jdbc").option("url", ds_url).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").option("encrypt", "false").option("trustServerCertificate", "false")\
    .option("user",     sa.target_db['username'])\
    .option("password", sa.target_db['password'])\
    .option("query",    tx_delete_query)\
    .outputMode(""))\




DELETE FROM tsl_psa_yahoo_exchange_rate.tsl_eur_x_usd
jdbc:sqlserver://misset.synology.me:1433;databaseName=meta


AttributeError: 'DataFrameReader' object has no attribute 'outputMode'

In [89]:
spark.truncateTable("temp_table").option("url", ds_url).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").option("encrypt", "false").option("trustServerCertificate", "false")\
    .option("user",     sa.target_db['username'])\
    .option("password", sa.target_db['password'])
    

AttributeError: 'SparkSession' object has no attribute 'truncateTable'

In [84]:

spark.sql(f"TRUNCATE TABLE {nm_target_schema}.{nm_target_table}")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `tsl_psa_yahoo_exchange_rate`.`tsl_eur_x_usd` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 15;
'TruncateTable
+- 'UnresolvedTable [tsl_psa_yahoo_exchange_rate, tsl_eur_x_usd], TRUNCATE TABLE


In [ ]:
# Load Source DataFrame to SQL Schema / Table
result = source_df.write.format("jdbc").option("url", ds_url) \
    .option("schema",  nm_target_schema) \
    .option("dbtable", nm_target_table) \
    .mode("append").save()

# Show Input Parameter(s)
if (is_debugging == "1"):
    print(f"nm_target_schema : '{nm_target_schema}'")
    print(f"nm_target_table  : '{nm_target_table}'")
    print(f"ni_ingested      : # {str(result)}")

In [ ]:
# If is Debugging then show imput parameters
if (is_debugging == 1):
    print("wtb_1_any_ds_url   : '" + wtb_1_any_ds_url + "'")
    print("wtb_2_any_ds_path  : '" + wtb_2_any_ds_path + "'")
    print("wtb_3_any_ni_index : '" + wtb_3_any_ni_index + "'")
    print("DataFrame:")
    df.show(10)